In [ ]:
import csv
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Float, func


In [ ]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [ ]:
# Define the dental_units table
class Unit(Base):
    __tablename__ = 'units'
    id = Column(Integer, primary_key=True)
    lab_name = Column(String)
    day = Column(String)
    nb_cases = Column(Integer)
    nb_units = Column(Integer)

In [ ]:
Base.metadata.tables

In [ ]:
# Create our database engine
engine = create_engine('sqlite:///labs_data.sqlite')

In [ ]:
# Create our tables in the database
Base.metadata.create_all(engine)

In [ ]:
# The ORM’s “handle” to the database is the Session.
from sqlalchemy.orm import Session
session = Session(engine)

In [ ]:
with open('data/data_time_series_analysis_forecasting_daily.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        session.add(Unit(lab_name=row['lab_name'], day=row['day'], nb_cases=row['nb_cases'],
                         nb_units=row['nb_units']))

In [ ]:
engine.execute('select * from units').fetchall()

In [ ]:
session.new

In [ ]:
session.commit()

In [ ]:
session.new

In [ ]:
# query the database
lab_name = '3DDENTALLABORATORIES'
session.query(Unit.day, Unit.nb_cases, Unit.nb_units).filter(Unit.lab_name == lab_name).all()

In [ ]:
len(results)

# Open database and test queries

In [2]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [3]:
engine = create_engine('sqlite:///labs_data.sqlite')
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['units']

In [4]:
Units = Base.classes.units

In [5]:
session = Session(engine)

In [6]:
sel = [func.strftime("%Y-%m", Units.day), func.sum(Units.nb_cases), func.sum(Units.nb_units)]
results = session.query(*sel).group_by(func.strftime("%Y-%m", Units.day)).all()
results


[('2011-07', 128, 150),
 ('2011-08', 108, 138),
 ('2011-09', 89, 103),
 ('2011-10', 69, 85),
 ('2011-11', 71, 98),
 ('2011-12', 68, 95),
 ('2012-01', 189, 278),
 ('2012-02', 726, 1063),
 ('2012-03', 898, 1304),
 ('2012-04', 818, 1302),
 ('2012-05', 1472, 2259),
 ('2012-06', 2654, 4106),
 ('2012-07', 2394, 3656),
 ('2012-08', 2584, 3921),
 ('2012-09', 2662, 3904),
 ('2012-10', 4593, 6654),
 ('2012-11', 5293, 7424),
 ('2012-12', 4994, 7337),
 ('2013-01', 7511, 10266),
 ('2013-02', 7148, 9722),
 ('2013-03', 8006, 10893),
 ('2013-04', 8487, 11889),
 ('2013-05', 8863, 12202),
 ('2013-06', 8523, 12096),
 ('2013-07', 9153, 12867),
 ('2013-08', 9156, 12804),
 ('2013-09', 9080, 13119),
 ('2013-10', 13072, 19224),
 ('2013-11', 11978, 17970),
 ('2013-12', 12310, 18563),
 ('2014-01', 16961, 25017),
 ('2014-02', 17766, 26201),
 ('2014-03', 19692, 29695),
 ('2014-04', 20676, 31038),
 ('2014-05', 22168, 34165),
 ('2014-06', 23676, 36052),
 ('2014-07', 23772, 36041),
 ('2014-08', 23045, 35158),
 ('201

In [7]:
len(results)

95

In [8]:
df = pd.DataFrame(results, columns=['month', 'nb_cases', 'nb_units'])
df.tail()

,month,nb_cases,nb_units
90,2019-01,193559,312134
91,2019-02,179632,292222
92,2019-03,198692,324640
93,2019-04,206121,332745
94,2019-05,28172,45003


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 3 columns):
month       95 non-null object
nb_cases    95 non-null int64
nb_units    95 non-null int64
dtypes: int64(2), object(1)
memory usage: 2.3+ KB


In [10]:
df.tail()

,month,nb_cases,nb_units
90,2019-01,193559,312134
91,2019-02,179632,292222
92,2019-03,198692,324640
93,2019-04,206121,332745
94,2019-05,28172,45003


In [11]:
df.month = pd.to_datetime(df.month)
df.set_index(['month'], inplace=True)
df.tail()

,nb_cases,nb_units
month,,
2019-01-01,193559,312134
2019-02-01,179632,292222
2019-03-01,198692,324640
2019-04-01,206121,332745
2019-05-01,28172,45003


In [12]:
df.at['2019-05-01', 'nb_cases'] = 194501 # average of first 4 months in 2019
df.at['2019-05-01', 'nb_units'] = 315435 # average of first 4 months in 2019
df.tail()

,nb_cases,nb_units
month,,
2019-01-01,193559,312134
2019-02-01,179632,292222
2019-03-01,198692,324640
2019-04-01,206121,332745
2019-05-01,194501,315435


In [13]:
df.reset_index(inplace=True)
df.head()

,month,nb_cases,nb_units
0,2011-07-01,128,150
1,2011-08-01,108,138
2,2011-09-01,89,103
3,2011-10-01,69,85
4,2011-11-01,71,98


In [14]:
df.month = pd.to_datetime(df.month)
df.set_index(['month'], inplace=True)
df.index.name=None
df.head()

,nb_cases,nb_units
2011-07-01,128,150
2011-08-01,108,138
2011-09-01,89,103
2011-10-01,69,85
2011-11-01,71,98


In [15]:
import statsmodels.api as sm
from datetime import datetime
from dateutil.relativedelta import relativedelta

/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [16]:
model_cases = sm.tsa.statespace.SARIMAX(df.nb_cases, trend='c', order=(0,1,1), seasonal_order=(0,0,1,12), enforce_stationarity=False, enforce_invertibility=False)
results_cases = model_cases.fit
model_units = sm.tsa.statespace.SARIMAX(df.nb_units, trend='c', order=(1,0,0), seasonal_order=(0,1,1,12), enforce_stationarity=False, enforce_invertibility=False)
results_units = model_units.fit()


In [17]:
start = datetime.strptime("2019-06-01", "%Y-%m-%d")
date_list = [start + relativedelta(months=x) for x in range(0,12)]
future = pd.DataFrame(index=date_list, columns= df.columns)
df = pd.concat([df, future])
future

,nb_cases,nb_units
2019-06-01,NaN,NaN
2019-07-01,NaN,NaN
2019-08-01,NaN,NaN
2019-09-01,NaN,NaN
2019-10-01,NaN,NaN
2019-11-01,NaN,NaN
2019-12-01,NaN,NaN
2020-01-01,NaN,NaN
2020-02-01,NaN,NaN
2020-03-01,NaN,NaN


In [18]:
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.seasonal import seasonal_decompose
fm = 12
df['forecast_cases'] = results_cases.predict(start = 83, end = 94+int(fm), dynamic= True)  
df['forecast_units'] = results_units.predict(start = 83, end = 94+int(fm), dynamic= True)  


AttributeError: 'function' object has no attribute 'predict'

In [19]:
print(results_cases.summary())

AttributeError: 'function' object has no attribute 'summary'